# Music Listening Habits Data Mining

In [3]:
import pandas as pd
import os
import json

## Data needs to support vizualisation

- Top 15 artists (name & link) ✅
- Listening duration per artist ✅
- Number of different tracks listened per artist ✅

- Top 15 genres for each user ✅
- Listening time for each genre ✅
- Proportion of listening per genre ✅

- Average listening time per user ✅

- Ranking of genres per period (each week, each months)
- Ranking of tracks per period (each week, each months)✅
- Ranking of artist per period  (each week, each months)✅

- Exact listening time per user per date

## Import Data for each users

In [60]:
df_clement = pd.read_excel("./data/Archive-Clement-Deezer/clement-deezer-data.xlsx", "10_listeningHistory")

In [55]:
df_clement.head(5)

,Song Title,Artist,ISRC,Album Title,IP Address,Listening Time,Platform Name,Platform Model,Date
0,Beggin,Madcon,NOBLV0703011,Beggin,37.165.218.4,1,android,HUAWEI ANE-LX1,2022-03-08 13:49:04
1,Beggin,Madcon,NOBLV0703011,Beggin,37.166.33.146,218,android,HUAWEI ANE-LX1,2022-02-22 16:19:01
2,Beggin,Madcon,NOBLV0703011,Beggin,37.166.68.99,218,android,samsung SM-A528B,2023-11-11 11:07:36
3,Beggin,Madcon,NOBLV0703011,Beggin,193.51.159.240,4,android,HUAWEI ANE-LX1,2022-04-20 09:07:38
4,Beggin,Madcon,NOBLV0703011,Beggin,37.169.42.32,218,android,HUAWEI ANE-LX1,2022-02-27 18:53:00


In [54]:
df_clement.count("")

Song Title        36276
Artist            36276
ISRC              36276
Album Title       36276
IP Address        36276
Listening Time    36276
Platform Name     36276
Platform Model    31978
Date              36276
dtype: int64

## Retrieve useful columns

In [18]:
df_clement = df_clement[["Song Title", "Artist", "Album Title", "Date"]]


## Columns format

In [56]:
df_clement = df_clement.sort_values("Date")
df_clement['Date'] = pd.to_datetime(df_clement['Date'])

In [59]:
(df_clement.head
(1))

,Song Title,Artist,ISRC,Album Title,IP Address,Listening Time,Platform Name,Platform Model,Date
9366,Mme Pavoshko,Black M,FR7U61200382,Le monde plus gros que mes yeux,88.170.1.13,256,android,NaN,2015-06-21 18:15:59


## Explore within time interval

In [41]:
# Define the date interval
start_date = "2024-01-01"
end_date = "2025-01-31"

# Filter the DataFrame
date_filtered_df = df_clement[(df_clement['Date'] >= start_date) & (df_clement['Date'] <= end_date)]

In [42]:
artist_counts = date_filtered_df.groupby('Artist').size().reset_index(name='Total Count')

In [43]:
artist_counts.sort_values("Total Count", ascending=False).head(15)

,Artist,Total Count
141,Imagine Dragons,292
129,Grandson,261
296,Starset,168
186,Linkin Park,138
313,System of a Down,115
308,Sub Urban,105
347,Three Days Grace,101
131,Green Day,95
137,Hollywood Undead,92
103,Escape the Fate,91


---- 

In [70]:
df_clement = pd.read_excel("./data/Archive-Clement-Deezer/clement-deezer-data.xlsx", "10_listeningHistory")
df_clement = df_clement.sort_values("Date")
df_clement['Date'] = pd.to_datetime(df_clement['Date'])
artist_counts = df_clement.groupby('Artist').size().reset_index(name='Total Count').sort_values("Total Count", ascending=False)


In [71]:
artist_counts

,Artist,Total Count
483,Imagine Dragons,1531
443,Grandson,804
1139,System of a Down,757
1226,Three Days Grace,629
672,Linkin Park,530
...,...,...
255,Daniele Leoni,1
791,Missing Persons,1
795,Moby,1
796,Modern Machines,1


,Artist,Total Count
0,100 Ans De Musiques De Films,1
1,"113, Magic System, Mohamed Lamine",1
2,12 Stones,4
3,2010s Karaoke Band,1
4,2CELLOS,5
...,...,...
1353,soundssdeep,1
1354,the EPM project,46
1355,youtunes,5
1356,Ødyssey,26


In [83]:
timespans = [
    ("4 weeks", ('2024-11-22', '2024-12-22')),
    ("3 months", ('2024-09-22', '2024-12-22')),
    ("6 months", ('2024-06-22', '2024-12-22')),
    ("1 year", ('2023-11-22', '2024-12-22')),
    ("all time", ('2002-01-01', '2024-12-22'))
]

In [84]:
original_clement = pd.read_excel("./data/Archive-Clement-Deezer/clement-deezer-data.xlsx", "10_listeningHistory")

original_clement = df_clement[["Song Title", "Artist", "Album Title", "Date"]]
original_clement = original_clement.sort_values("Date")
original_clement['Date'] = pd.to_datetime(original_clement['Date'])


for timespan in timespans:
    print(timespan)
    copy = original_clement.copy()
    
    copy = copy[(copy['Date'] >= timespan[1][0]) & (copy['Date'] <= timespan[1][1])]
    
    df_top_songs = df_top_artist = copy.groupby('Song Title').size().reset_index(name='Total Count').sort_values('Total Count', ascending=False).head(15)
    
    df_top_artists = copy.groupby('Artist').size().reset_index(name='Total Count').sort_values('Total Count', ascending=False).head(15)
    
    df_top_albums = copy.groupby('Album Title').size().reset_index(name='Total Count').sort_values('Total Count', ascending=False).head(15)
    
    df_top_genres = None
    
    print(df_top_artists, df_top_songs, df_top_albums)
    
    
    
    
    

('4 weeks', ('2024-11-22', '2024-12-22'))
                       Artist  Total Count
86                       d4vd           21
83                    Woodkid           20
62        Royal & the Serpent           19
27                  FEVER 333           15
6                    Ashnikko           15
82          Twenty One Pilots           14
45                 Mako, Grey           13
84                       Zand           13
50              Misha Mansoor           13
61  Raja Kumari, Stefflon Don           13
4   Arcane, League of Legends           12
28              Freya Ridings           12
71             Stromae, Pomme           12
40              King Princess           12
23                 Eason Chan           11                                            Song Title  Total Count
89  To Ashes and Blood (from the series Arcane Lea...           20
93  Wasteland (from the series Arcane League of Le...           16
35  Hellfire (from the series Arcane League of Leg...           15
57

--- 

# Matt's data


In [56]:
df_matthieu1 = pd.read_json("data/Archive-Matthieu-Spotify/Spotify Account Data/StreamingHistory_music_0.json")
df_matthieu2 = pd.read_json("data/Archive-Matthieu-Spotify/Spotify Account Data/StreamingHistory_music_1.json")
df_matthieu3 = pd.read_json("data/Archive-Matthieu-Spotify/Spotify Account Data/StreamingHistory_music_2.json")

df_matthieu = pd.concat([df_matthieu1, df_matthieu2, df_matthieu3])

In [53]:
df_matthieu["endTime"] = pd.to_datetime(df_matthieu['endTime'])
df_matthieu = df_matthieu.sort_values("endTime", ascending=True)
df_matthieu['msPlayed'] = (df_matthieu["msPlayed"] / 100).astype(int)

In [55]:
df_matthieu.groupby('artistName').size().reset_index(name='Total Count').sort_values('Total Count', ascending=False)

,artistName,Total Count
833,Forrest Frank,888
1836,NF,664
2566,Tiffany Hudson,531
2358,Sofiane Pamart,439
724,Elevation Worship,420
...,...,...
1331,KAMAMILK,1
1330,KAFFKIEZ,1
1329,K3ndrick,1
1328,K2S,1


In [57]:
df_matthieu

,endTime,artistName,trackName,msPlayed
0,2023-09-17 19:40,AJR,Sober Up (feat. Rivers Cuomo),4866
1,2023-12-04 17:07,Lecrae,They Ain’t Know,69652
2,2023-12-05 01:01,gio.,shadows,69115
3,2023-12-05 07:09,Young Oceans,You Are Not Far,134626
4,2023-12-05 07:13,Josiah Queen,Fishes and Loaves,216979
...,...,...,...,...
7807,2024-12-05 23:41,NF,How Could You Leave Us,322586
7808,2024-12-05 23:44,NF,Lie,209213
7809,2024-12-05 23:49,NF,HOPE,264473
7810,2024-12-05 23:53,NF,TRUST,264415


---

# API Request for genres